In [1]:
import pandas as pd
import os

In [2]:
# This function creates the path to the main Excel file of this project
# Uses os library methods to ensure the file path works in both local and GitHub CI/CD environments
def create_file_path(file_relative_path):
    # Store the xlsx file dir into a var
    dir = '../data'
    
    # Iterate under dir data to have all files
    for filename in os.listdir(dir):
        # Condition to confirm which file is == year_make_model_df.xlsx
        if filename.endswith(file_relative_path):
            # Join the dir + filename to create the path to year_make_model_df.xlsx file  
            xlsx_file_fullpath = os.path.join(dir, filename)
            # Print the result
            print(xlsx_file_fullpath)
    return xlsx_file_fullpath

In [3]:
# Read excel 
df = pd.read_excel(create_file_path('year_make_model_df.xlsx'))

../data\year_make_model_df.xlsx


In [4]:
# Create df
df_subaru = df[df["Make"] == "Subaru"]

In [5]:
# Check the dataframe to see how it is
df_subaru.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 1334 to 3387
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            53 non-null     int64 
 1   Make            53 non-null     object
 2   Model           53 non-null     object
 3   Security        46 non-null     object
 4   ParameterReset  0 non-null      object
dtypes: int64(1), object(4)
memory usage: 2.5+ KB


In [5]:
# Function to create list with unique items from given column
def data_from_column(df, column):
    # List to append the items
    data_column_list =[]

    # Iterate over given column
    for data in df[column]:
        # Append the items from the given column
        data_column_list.append(data)

    # Return a set with unique items from given column
    return set(data_column_list)

In [6]:
# Call the function to create a set with unique items from column model
data_from_column(df_subaru, "Model")

{'B9 Tribeca (2006 – 2007)\nTribeca (2008 – onward)',
 'BRZ',
 'Baja',
 'Forester',
 'Impreza',
 'Legacy',
 'Outback',
 'XV Crosstrek'}

### Changes needed:
- Explode B9 Tribeca and Tribeca models in separate lines
- Remove the string (2006 - 2007) from model B9 Tribeca
- Remove the string (2008 - onward) from model Tribeca 

In [7]:
# Explode models B9 Tribeca and Tribeca in 2 separate lines
df_subaru_tribeca_models_exploded = df_subaru.assign(Model=df_subaru['Model'].str.split("\n")).explode("Model")

In [8]:
# Remove the string (2006 - 2007) from model B9 Tribeca
df_subaru_tribeca_models_exploded['Model'] = df_subaru_tribeca_models_exploded['Model'].str.replace('B9 Tribeca (2006 – 2007)', 'B9 Tribeca', regex=False)

In [9]:
# Remove the string (2008 - onward) from model Tribeca
df_subaru_tribeca_models_exploded['Model'] = df_subaru_tribeca_models_exploded['Model'].str.replace('Tribeca (2008 – onward)', 'Tribeca', regex=False)

In [10]:
# Call the funtion to create a set with unique models after changes
data_from_column(df_subaru_tribeca_models_exploded, 'Model')

{'B9 Tribeca',
 'BRZ',
 'Baja',
 'Forester',
 'Impreza',
 'Legacy',
 'Outback',
 'Tribeca',
 'XV Crosstrek'}

In [11]:
# Drop column ParameterReset
df_subaru_dropped_pr_column = df_subaru_tribeca_models_exploded.drop(columns=['ParameterReset'])

In [12]:
# Call the function to create a set with unique items from column security
data_from_column(df_subaru_dropped_pr_column, "Security")

{'Std 2.5\nXT',
 'Std 2.5\nXT &\n3.0 R',
 'Std 2.5 GT',
 'Std 2.5 XT',
 'Std WRX\nSTI',
 'Std all\nvariants',
 'Std all Variants',
 'Std all WRX &\nWRX STI',
 'Std all variants',
 nan}

CHANGES NEEDED:
- Fill out null values.
- Replace the charactere '\n' by space ' '.

In [13]:
# Fill out null values
df_subaru_dropped_pr_column['Security'].fillna('Information not available', inplace=True)

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_2884\1839461870.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_subaru_dropped_pr_column['Security'].fillna('Information not available', inplace=True)


In [14]:
# Replace the charactere '\n' by space ' '
df_subaru_dropped_pr_column['Security'] = df_subaru_dropped_pr_column['Security'].str.replace('\n', ' ', regex=False)

In [15]:
# Function to check if there are duplicated values
def check_duplicated_values(df):
    # Create a column with bool values showing whether or not there are duplicated values
    df['Dups'] = df.duplicated()
    # Return bool values counted
    return df['Dups'].value_counts()

In [ ]:
# Call the function to check dup values
check_duplicated_values(df_subaru_dropped_pr_column)

Dups
False    61
Name: count, dtype: int64

In [ ]:
# Export the to .csv file
# df_subaru_dropped_pr_column.to_csv('C:\\Language_Projects\\Language_Projects\\Python\\Flagship_1\\vehicle_security_system_data_cleaning\\data\\df_subaru.csv', index=False)